In [1]:
using Pkg; Pkg.activate("..")
using GraphFusedElasticNet
using ProgressMeter
using Distributions
using Plots
using StatsFuns

 Activating environment at `C:\Users\mauri\Github\GraphFusedElasticNet.jl\Project.toml`
┌ Info: Precompiling GraphFusedElasticNet [f5121390-0a77-11ea-346e-a5df2c1e4685]
└ @ Base loading.jl:1278


Generate fake data from a 3D grid.

In [2]:
nr = 61
nc = 61
nk = 30

p0 = zeros(nr, nc, nk)
s = zeros(nr, nc, nk)
a = zeros(nr, nc, nk)

for k in 1:nk
    for j in 1:nc
        for i in 1:nr
            b = Float64((i - (nr / 2)) * (j - (nc / 2)) * (k - nk / 2) ≥ 0)
            δk = (0.5 / nk) * (k ÷ (nk ÷ 5))
            p0[i, j, k] = 0.7 * b + 0.3 * (1 - b)
            a[i, j, k] = rand([1, 5.])
            s[i, j, k] = rand(Binomial(a[i, j, k], p0[i, j, k]))
        end
    end
end

p_mle = ((s .+ 1e-6) ./ (a .+ 2e-6))


# build graph
edges = Set{Tuple{Int, Int}}()
index_map = reshape(1:(nr * nc * nk), (nr, nc, nk));


sorted_tuple(i, j) = (min(i, j), max(i, j))

for k in 1:nk
    for j in 1:nc
        for i in 1:nr
            nbrs_ = Int[]
            (i > 1) && push!(nbrs_, index_map[i - 1, j, k])
            (i < nr) && push!(nbrs_, index_map[i + 1, j, k])
            (j > 1) && push!(nbrs_, index_map[i, j - 1, k])
            (j < nc) && push!(nbrs_, index_map[i, j + 1, k])
            (k > 1) && push!(nbrs_, index_map[i, j, k - 1])
            (k < nk) && push!(nbrs_, index_map[i, j, k + 1])

            src = index_map[i, j, k]
            for tgt in nbrs_
                push!(edges, sorted_tuple(src, tgt))
            end
        end
    end
end

edges = collect(edges)
s_vec = vec(s);
a_vec = vec(a);

In [3]:
print("Graph with $(nr * nc * nk) nodes and $(length(edges)) edges")

Graph with 111630 nodes and 327509 edges

In [4]:
# anim = @animate for k in 1:nk
#     p1 = plot(Gray.(p0[:, :, k]), title="signal");
#     p2 = plot(Gray.(p_mle[:, :, k]), title="data");
#     plot(p1, p2, axis=nothing, layout=(1, 2), figsize=(500, 200));
# end
# gif(anim, "3dgrid.gif", fps = 15);

![](3dgrid.gif)

In [5]:
mod = BayesianBinomialGFEN(edges, tv1=5.0)

BayesianBinomialGFEN

In [7]:
init = logistic.(vec(p_mle))
@time results = sample_chain(mod, s_vec, a_vec, 2_500, init=init, async=true)

Progress:   3%|██                                       |  ETA: 0:54:12

LoadError: InterruptException: